In [1]:
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1
!pip install --upgrade ibm-aigov-facts-client | tail -n 1
!pip install --upgrade evaluate --no-cache | tail -n 1
!pip install --upgrade rouge_score --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1
!pip install --upgrade torchmetrics --no-cache | tail -n 1
!pip install --upgrade pyspellchecker spacy --no-cache | tail -n 1

In [2]:
import spacy
spacy.cli.download("en_core_web_sm")
!python -m nltk.downloader punkt
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 98.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
use_cpd = False
CLOUD_API_KEY = "xxx"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator,CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

if use_cpd:
    authenticator = CloudPakForDataAuthenticator(
            url=WOS_CREDENTIALS['url'],
            username=WOS_CREDENTIALS['username'],
            apikey=WOS_CREDENTIALS['apikey'],
            disable_ssl_verification=True,
        )
    
    client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
    print(client.version)
else:
    authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
    client = APIClient(authenticator=authenticator)
    print(client.version)

3.0.40


In [5]:
PROJECT_ID = "ab37b074-2174-4cba-ab6c-8545266e3cf2" # YOUR_PROJECT_ID

In [6]:
use_existing_space = True # Set it as False if user wants to create a new space

In [7]:
import json
from ibm_watson_machine_learning import APIClient
WML_CREDENTIALS = {
                "url": "https://us-south.ml.cloud.ibm.com",
                "apikey": CLOUD_API_KEY
}
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.360'

In [8]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------------  ------------------------
ID                                    NAME                  CREATED
066d6f31-9f63-4498-aa17-cf8af695bac8  niklas-test           2024-09-12T15:08:59.866Z
c5426b24-db66-4ed8-a0dd-bfde5e057457  niklas-claim-summary  2024-09-12T09:49:34.381Z
f9c9a838-28b0-4504-ba7d-d8e303ea1bae  watsonx-demo          2024-09-10T15:59:37.648Z
------------------------------------  --------------------  ------------------------


,ID,NAME,CREATED
0,066d6f31-9f63-4498-aa17-cf8af695bac8,niklas-test,2024-09-12T15:08:59.866Z
1,c5426b24-db66-4ed8-a0dd-bfde5e057457,niklas-claim-summary,2024-09-12T09:49:34.381Z
2,f9c9a838-28b0-4504-ba7d-d8e303ea1bae,watsonx-demo,2024-09-10T15:59:37.648Z


In [9]:
space_id = "c5426b24-db66-4ed8-a0dd-bfde5e057457"

In [10]:
WML_INSTANCE_NAME =  "Watson Machine Learning" # YOUR_WML_INSTANCE_NAME
WML_CRN =  "crn:v1:bluemix:public:pm-20:us-south:a/000e0d2204334dc99d336a3bc0cd99be:4f9ee7b9-c9d2-446a-9de1-1e3ca2725e98::"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/000e0d2204334dc99d336a3bc0cd99be:37b295a1-3e57-4241-87e6-67724af28139::"

In [11]:
wml_client.set.default_space(space_id)
print(space_id)

c5426b24-db66-4ed8-a0dd-bfde5e057457


In [12]:
import requests, json
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY,
        "response_type": "cloud_iam"
    }
    response = requests.post("https://iam.cloud.ibm.com/identity/token", data=data, headers=headers)
    json_data = response.json()
    iam_access_token = json_data["access_token"]
    return iam_access_token

iam_access_token = generate_access_token()

In [13]:
!pip install --upgrade ibm-aigov-facts-client | tail -n 1

from ibm_aigov_facts_client import AIGovFactsClient

facts_client = AIGovFactsClient(
    api_key=CLOUD_API_KEY,
    container_id=PROJECT_ID,
    container_type="project",
    disable_tracing=True
)

In [14]:
from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate

detached_information = DetachedPromptTemplate(
    prompt_id="detached_prompt",
    model_id="google/flan-t5-base",
    model_provider="Hugging Face",
    model_name="google/flan-t5-base",
    model_url="https://huggingface.co/google/flan-t5-base",
    prompt_url="prompt_url",
    prompt_additional_info={"model_owner": "huggingface"}
)

task_id = "summarization"
name = "External prompt sample with custom metrics"
description = "My custom metrics detached prompt"
model_id = "google/flan-t5-base"

# define parameters for PromptTemplate
prompt_variables = {"original_text": ""}
input = "{original_text}"
input_prefix= "Input:"
output_prefix= "Output:"

prompt_template = PromptTemplate(
    input=input,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix
)

pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

2024/09/19 14:56:40 INFO : ------------------------------ Detached Prompt Creation Started ------------------------------
2024/09/19 14:56:40 INFO : The detached prompt with ID 8e511707-8de5-4410-abb8-4374c9e9fe05 was created successfully in container_id ab37b074-2174-4cba-ab6c-8545266e3cf2.


In [15]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

SERVICE_URL = "https://api.aiopenscale.cloud.ibm.com"
IAM_URL = "https://iam.cloud.ibm.com"

service_instance_id = None # Update this to refer to a particular service instance
authenticator = IAMAuthenticator(
    apikey=CLOUD_API_KEY,
    url=IAM_URL
)
wos_client = APIClient(
    authenticator=authenticator,
    service_url=SERVICE_URL,
    service_instance_id=service_instance_id
)
data_mart_id = wos_client.service_instance_id
print(wos_client.version)

3.0.41


In [16]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
   
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
   
    return None
 
monitor_name = 'Custom metrics with detached prompt template'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]
 
existing_definition = get_definition(monitor_name)
 
if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition

In [17]:
custom_monitor_id = custom_monitor_details.metadata.id
 
print(custom_monitor_id)


custom_metrics_with_detached_prompt_template


In [18]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)


Monitor definition details: {
  "metadata": {
    "id": "custom_metrics_with_detached_prompt_template",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/000e0d2204334dc99d336a3bc0cd99be:cb0dce18-46f9-4618-a077-4e2c8dbdf57b:monitor_definition:custom_metrics_with_detached_prompt_template",
    "url": "/v2/monitor_definitions/custom_metrics_with_detached_prompt_template",
    "created_at": "2024-09-19T11:57:39.821000Z",
    "created_by": "IBMid-0600000RGH"
  },
  "entity": {
    "name": "Custom metrics with detached prompt template",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "incre

In [19]:
label_column = "reference_summary"
operational_space_id = "development"
problem_type = "summarization"
input_data_type = "unstructured_text"
monitors = {}

response = wos_client.monitor_instances.mrm.execute_prompt_setup(
    prompt_template_asset_id=project_pta_id, 
    project_id=PROJECT_ID,
    label_column=label_column,
    operational_space_id=operational_space_id, 
    problem_type=problem_type,
    input_data_type=input_data_type, 
    supporting_monitors=monitors, 
    background_mode=False
)

result = response.result
result.to_dict()

This method will be deprecated in the next release and be replaced by wos_client.wos.execute_prompt_setup() method



 Waiting for end of adding prompt setup 8e511707-8de5-4410-abb8-4374c9e9fe05 




running...
finished

---------------------------------------------------------------
 Successfully finished setting up prompt template subscription 
---------------------------------------------------------------




{'prompt_template_asset_id': '8e511707-8de5-4410-abb8-4374c9e9fe05',
 'project_id': 'ab37b074-2174-4cba-ab6c-8545266e3cf2',
 'deployment_id': 'fe9806e4-14b2-49b2-8073-3e24d76b2bbd',
 'service_provider_id': '37e697de-718a-4f10-b4b5-e3dec9fa6338',
 'subscription_id': '4ee7b95d-7088-44f8-bac8-6ebc99f6d944',
 'mrm_monitor_instance_id': '70d6239a-1e0f-492d-a8a8-b64878356e5e',
 'start_time': '2024-09-19T14:57:05.535375Z',
 'end_time': '2024-09-19T14:57:32.220848Z',
 'status': {'state': 'FINISHED'}}

In [20]:
response = wos_client.monitor_instances.mrm.get_prompt_setup(
    prompt_template_asset_id=project_pta_id,
    project_id=PROJECT_ID
)

result = response.result
result_json = result.to_dict()

if result_json["status"]["state"] == "FINISHED":
    print("Finished prompt setup. The response is {}".format(result_json))
else:
    print("Prompt setup failed. The response is {}".format(result_json))

This method will be deprecated in the next release and be replaced by wos_client.wos.get_prompt_setup() method
Finished prompt setup. The response is {'prompt_template_asset_id': '8e511707-8de5-4410-abb8-4374c9e9fe05', 'project_id': 'ab37b074-2174-4cba-ab6c-8545266e3cf2', 'deployment_id': 'fe9806e4-14b2-49b2-8073-3e24d76b2bbd', 'service_provider_id': '37e697de-718a-4f10-b4b5-e3dec9fa6338', 'subscription_id': '4ee7b95d-7088-44f8-bac8-6ebc99f6d944', 'mrm_monitor_instance_id': '70d6239a-1e0f-492d-a8a8-b64878356e5e', 'start_time': '2024-09-19T14:57:05.535375Z', 'end_time': '2024-09-19T14:57:32.220848Z', 'status': {'state': 'FINISHED'}}


In [21]:
subscription_id = result_json["subscription_id"]
mrm_monitor_instance_id = result_json["mrm_monitor_instance_id"]

In [22]:
data_mart_id = wos_client.service_instance_id
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )
 
thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]
parameters = {
   "metric_id": "custom_metrics_example"
}
custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target,
            parameters=parameters
).result





 Waiting for end of monitor instance creation a53760e9-b273-439d-9e87-0d214b2366a2 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [23]:
wos_client.monitor_instances.show(target_target_id=subscription_id)

cb0dce18-46f9-4618-a077-4e2c8dbdf57b,active,4ee7b95d-7088-44f8-bac8-6ebc99f6d944,subscription,custom_metrics_with_detached_prompt_template,2024-09-19 14:58:08.603000+00:00,a53760e9-b273-439d-9e87-0d214b2366a2
cb0dce18-46f9-4618-a077-4e2c8dbdf57b,active,4ee7b95d-7088-44f8-bac8-6ebc99f6d944,subscription,model_health,2024-09-19 14:57:21.735000+00:00,a0a6e623-9725-421d-80ef-dde755453651
cb0dce18-46f9-4618-a077-4e2c8dbdf57b,active,4ee7b95d-7088-44f8-bac8-6ebc99f6d944,subscription,mrm,2024-09-19 14:57:24.125000+00:00,70d6239a-1e0f-492d-a8a8-b64878356e5e


In [26]:
#subscription.monitoring.get_details(monitor_uid=monitor_uid)
custom_monitor_instance_id = "a53760e9-b273-439d-9e87-0d214b2366a2"


In [27]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = "11122223333111abc"
#custom_monitoring_run_id = custom_monitor_instance_id
#measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc),
#                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc),
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}])]
print(measurement_request[0])
 
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)


{
  "timestamp": "2024-09-19T14:58:52.870170Z",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}
[{'measurement_id': '9dd026f1-a292-41ca-883c-4d06f16b6830', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'timestamp': '2024-09-19T14:58:52.870170Z'}]


In [31]:
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "9dd026f1-a292-41ca-883c-4d06f16b6830",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/000e0d2204334dc99d336a3bc0cd99be:cb0dce18-46f9-4618-a077-4e2c8dbdf57b:measurement:9dd026f1-a292-41ca-883c-4d06f16b6830",
    "url": "/v2/monitor_instances/a53760e9-b273-439d-9e87-0d214b2366a2/measurements/9dd026f1-a292-41ca-883c-4d06f16b6830",
    "created_at": "2024-09-19T14:58:55.063636Z",
    "created_by": "IBMid-0600000RGH"
  },
  "entity": {
    "timestamp": "2024-09-19T14:58:52.870170Z",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.8
          },
          {
            "id": "specificity",
            "value": 0.78,
            "lower_limit": 0.75
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "target": {
      "target_type": "sub